In [1]:
import numpy as np
import pandas as pd

from libreco.data import DatasetPure
from libreco.algorithms import BPR, WideDeep
from libreco.evaluation import evaluate

from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
!pip install numpy pandas scikit-learn tqdm scipy openai

In [3]:
! pip install -U LibRecommender

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocess Data

In [9]:
books_df = pd.read_csv('/content/drive/MyDrive/Books_Extended.csv', delimiter=';', low_memory=False)
ratings_df = pd.read_csv('/content/drive/MyDrive/Ratings.csv', delimiter=';')
users_df = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')

<ipython-input-9-497a94a7983d>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')


In [10]:
ratings_df.columns = ['user_id', 'item_id', 'label']

In [8]:
ratings_df = ratings_df[ratings_df['label'] != 0]

In [9]:
book_str_to_int = {book_id: i for i, book_id in enumerate(ratings_df['item_id'].unique())}
user_str_to_int = {user_id: i for i, user_id in enumerate(ratings_df['user_id'].unique())}

ratings_df['item'] = ratings_df['item_id'].map(book_str_to_int).to_numpy()
ratings_df['user'] = ratings_df['user_id'].map(user_str_to_int).to_numpy()

In [2]:
df_clened_with_emb = pd.read_csv('/content/df_with_emb_cleaned.csv')

# Train / Eval Split

In [3]:
train_df, eval_df = train_test_split(df_clened_with_emb, test_size=0.2, random_state=0)

In [6]:
eval_user_mask = eval_df['user'].isin(train_df['user'].unique())
eval_item_mask = eval_df['item'].isin(train_df['item'].unique())

eval_df = eval_df[eval_user_mask & eval_item_mask]

# 'user', 'item' must be the first two columns of the dataframe
train_df = train_df[columns]
eval_df = eval_df[columns]

In [4]:
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

In [5]:
columns = ['user', 'item', 'label']
user_cols = ['Age']
item_cols = pca_columns + ['Year']

columns = columns + user_cols + item_cols

In [7]:
from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

# Bayesian Personalized Ranking and Wide Deep


In [8]:
embedding_size = 32
n_epochs_bpr = 400
n_epochs_wide = 400
learning_rate = 0.01
batch_size = 256
random_seed = 42

In [10]:
# init the WideDeep model
wide_deep = WideDeep(
    task="ranking",
    data_info=data_info,
    embed_size=embedding_size,
    n_epochs=n_epochs_wide,
    lr={'wide': learning_rate, 'deep': learning_rate/10},
    batch_size=batch_size,
    use_bn=True,
    hidden_units=[64, 32],
    reg=None,
    dropout_rate=None,
    num_neg=1,
    seed=random_seed
)

In [11]:
# train WideDeep model
#from tensorflow.compat.v1 import reset_default_graph
#reset_default_graph()
wide_deep.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=test_data,

    metrics=["ndcg", "precision"]
)

/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.


Training start time: 2024-08-11 18:47:36


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)


total params: 281,995 | embedding params: 250,205 | network params: 31,790


train: 100%|██████████| 65/65 [00:00<00:00, 90.70it/s] 


...

	 eval ndcg@10: 0.0193
	 eval precision@10: 0.0031


train: 100%|██████████| 65/65 [00:00<00:00, 189.40it/s]


Epoch 360 elapsed: 0.348s
	 train_loss: 0.0852


eval_listwise:  15%|█▍        | 33/227 [00:00<00:01, 136.66it/s]


KeyboardInterrupt: 

In [13]:
# evaluate the WideDeep model
wide_deep_eval_result = evaluate(wide_deep, test_data, neg_sampling=True, metrics=["ndcg", "precision", "recall"])
print(f"Evaluation Results (WideDeep):\n{wide_deep_eval_result}")

eval_listwise: 100%|██████████| 227/227 [00:01<00:00, 145.03it/s]

Evaluation Results (WideDeep):
{'ndcg': 0.014243517072857275, 'precision': 0.0026431718061674008, 'recall': 0.012546908141621798}
